In [ ]:
# !pip -q install langchain==X.X.XXX

In [4]:
import os
import sys
import warnings

from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI

from langchain.llms import OpenAI
from langchain.llms import CTransformers

In [ ]:
pip show langchain

In [ ]:
pip show pydantic

In [1]:
from pydantic import BaseModel, Field

class multiplyInput(BaseModel):
    """Input for multiply two numbers."""

    first_int: int = Field(..., first_int="The first number to be multiplied")
    second_int: int = Field(..., second_int="The second number to be multiplied")
    

In [2]:
import os
import json
import requests
from typing import Optional, Type
from langchain.agents import Tool
from langchain.tools import BaseTool
from pydantic import ValidationError

class MultiplyTool(BaseTool):
    name = "Multiply two numbers"
    description = "Useful for when you need to multiply two numbers"

    def _run(self, first_int: int, second_int: int):

        def multiply(first_int: int, second_int: int) -> int:
            """Multiply two integers together."""
            return first_int * second_int

        return multiply(first_int, second_int)
       
    def _arun(self, first_int: int, second_int: int):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = multiplyInput
   

multiply_tool_getter = MultiplyTool()

In [5]:
result = multiply_tool_getter._run(5,10)
print(result)

50


In [6]:
import os
import requests

# https://dictionarygame-api.azurewebsites.net/admin/9440341601894424/games
# https://dictionarygame-api.azurewebsites.net/gamestatus/2023051105424097922400/allgameresults

def get_players_scores(game_code: str):
    """Get players scores data from the online game DictionaryGame.net, and return the list of winners"""
    api_endpoint = "https://dictionarygame-api.azurewebsites.net/admin/9440341601894424/games"
    #header_dic = {"Authorization": f'Bearer {os.environ.get("PROXYCURL_API_KEY")}'}

    # response = requests.get(
    #     api_endpoint, params={"url": linkedin_profile_url}, headers=header_dic
    # )

    response = requests.get(api_endpoint)

    if response.status_code == 404:
        return ""

    data = response.json()
    # data = {
    #     k: v
    #     for k, v in data.items()
    #     if v not in ([], "", "", None)
    #     and k in ["gameScore"]
    # }
    
    # if data.get("groups"):
    #     for group_dict in data.get("gameScore"):
    #         group_dict.pop("profile_pic_url")

    return data

In [7]:
result = get_players_scores('123456')
print(result)

[{'createdat': '2024-04-18T16:57:50.700Z', 'gamecode': '2024041816575070046300'}, {'createdat': '2024-03-26T22:03:27.061Z', 'gamecode': '2024032622032706187200'}, {'createdat': '2024-02-03T20:44:47.153Z', 'gamecode': '2024020320444715349700'}, {'createdat': '2024-01-01T01:47:02.404Z', 'gamecode': '2024010101470240472200'}, {'createdat': '2024-01-01T01:36:01.837Z', 'gamecode': '2024010101360183706600'}, {'createdat': '2023-02-11T09:33:38.910Z', 'gamecode': '202302110933389102800'}, {'createdat': '2023-02-11T08:50:50.268Z', 'gamecode': '2023021108505026898600'}, {'createdat': '2023-02-10T05:22:46.585Z', 'gamecode': '2023021005224658535700'}, {'createdat': '2023-02-09T07:30:25.359Z', 'gamecode': '2023020907302535944300'}, {'createdat': '2023-02-09T07:29:48.089Z', 'gamecode': '202302090729480895100'}, {'createdat': '2023-02-09T06:41:08.288Z', 'gamecode': '2023020906410828831100'}, {'createdat': '2023-02-09T06:33:11.618Z', 'gamecode': '2023020906331161814500'}, {'createdat': '2023-02-09T06:

In [8]:
from pydantic import BaseModel, Field

class AdminGameListInput(BaseModel):
    """Input for admin game code."""

    gamecode: str = Field(..., gamecode="Admin code to retrive list of game codes")


In [9]:
import os
import requests
import json
from langchain.agents import Tool
from langchain.tools import BaseTool
from typing import Optional, Type

class AdminGameListTool(BaseTool):
    name = "Get all games created by the Game administrator"
    description = "Useful for when you need to get a list of all game codes created by the Game administrator from the online game DictionaryGame.net webgame"

    def _run(self, admin_code: str):
        
        def get_game_list(admin_code: str):            
            api_endpoint = f"https://dictionarygame-api.azurewebsites.net/admin/{admin_code}/games"            
            response = requests.get(api_endpoint)
            
            if response.status_code == 404:
                return ""
                
            # data = response.json()
            return response.json()        

        return get_game_list(admin_code)
        
    def _arun(self, admin_code: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = AdminGameListInput

admin_gamelist_getter = AdminGameListTool()

In [10]:
from pydantic import BaseModel
from typing import Dict, List

# class GameScoreModel(BaseModel):
#     gameScore: Dict[str, str]

class GameScoreItem(BaseModel):
    playername: str
    score: str
    
class Model(BaseModel):
    gameScore: List[GameScoreItem]


In [11]:
from pydantic import BaseModel, Field

class PlayersScoresInput(BaseModel):
    """Input for game code check."""

    gamecode: str = Field(..., gamecode="The game code to get the players scores of the game")


In [12]:
import os
import json
import requests
from langchain.agents import Tool
from langchain.tools import BaseTool
from pydantic import ValidationError

class PlayersScoresTool(BaseTool):
    name = "Get Players Scores"
    description = "Useful for when you need to get a list of Players Scores from the online game DictionaryGame.net webgame"

    def _run(self, game_code: str):
        
        def get_players_scores(game_code: str):            
            api_endpoint = f"https://dictionarygame-api.azurewebsites.net/gamestatus/{game_code}/allgameresults"        
            
            response = requests.get(api_endpoint)
            
            if response.status_code == 404:
                return ""
                
            data = response.json()
            data = {
                k: v
                for k, v in data.items()
                if v not in ([], "", "", None)
                and k in ["gameScore"]
            }        
            # return data

            # print(response.json())
            
            try:
                # Attempt to parse the response JSON into the Pydantic model
                # game_score_data = GameScoreModel(**response.json())
                # game_score_data = Model(**data)
                # return game_score_data.gameScore
                return json.dumps(data)
            except ValidationError as e:
                # Handle validation errors here (e.g., log or raise an exception)
                print(f"Validation error: {e}")
                return {}        

        return get_players_scores(game_code)
        
    def _arun(self, game_code: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = PlayersScoresInput
    

players_scores_getter = PlayersScoresTool()

In [13]:
result = players_scores_getter._run('2023021005224658535700')
print(result)

{"gameScore": [{"playername": "Keyla", "score": "90"}, {"playername": "Fred", "score": "50"}, {"playername": "Lucas", "score": "50"}]}


In [54]:
# # Set up the turbo LLM
# turbo_llm = ChatOpenAI(
#     temperature=0,
#     model_name='gpt-3.5-turbo'
# )

In [14]:
MODEL_PATH = 'mistral-7b-instruct-v0.2.Q4_K_M.gguf'  

In [15]:
config = {
    "max_new_tokens": 2048,
    "context_length": 4096,
    "repetition_penalty": 1.1,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 0.9,
    "stream": True,
    "threads": int(os.cpu_count() / 2)
}

In [16]:
llm = CTransformers(model=MODEL_PATH, config=config)

In [21]:
from langchain.agents import AgentType, initialize_agent
from langchain.prompts.chat import SystemMessagePromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

tools = [admin_gamelist_getter, players_scores_getter, multiply_tool_getter]

# tools = [players_scores_getter]

conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    # agent='openai-multi-functions',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
    # agent_type=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION
)

/home/fredsena/FredCodes/GIT/fred_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [22]:
fixed_prompt = '''You are an Assistant designed to be able to assist with a specific range of tasks, from answering simple questions to providing explanations and discussions on a specific and limited range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant should use the available tools for questions about these topics. If the tool does not returns data, just say that information was not found.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a specific and limited range of questions. 
Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a specific range of topics. Whether you need help with a specific question or just want to have a information about a particular topic, Assistant is here to assist.'''


In [23]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [24]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [25]:
conversational_agent("who won the game number: 2023021005224658535700? return only the name of the player and multiply the score by 10")

/home/fredsena/FredCodes/GIT/fred_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
.

```json
{
    "action": "Get Players Scores",
    "action_input": "2023021005224658535700"
}
```
Observation: {"gameScore": [{"playername": "Keyla", "score": "90"}, {"playername": "Fred", "score": "50"}, {"playername": "Lucas", "score": "50"}]}
Thought:

AI: The player named Keyla won the game with a score of 900 (90 multiplied by 10).

```json
{
    "action": "Final Answer",
    "action_input": "The player named Keyla won the game with a score of 900."
}
```
Confidence: 95%

> Finished chain.


{'input': 'who won the game number: 2023021005224658535700? return only the name of the player and multiply the score by 10',
 'chat_history': [],
 'output': 'The player named Keyla won the game with a score of 900.'}

In [ ]:
conversational_agent("based on the admin code: 9440341601894424 return the data, then search based on gamecode data who won each game")

In [ ]:
conversational_agent("game number: 2023021005224658535700? return only the name of the player and multiply the score by 10")